# 도매가 데이터 결측값 보간 전처리

### import

In [ ]:
import pandas as pd
import numpy as np
import datetime
import copy 

### 데이터셋 input

In [ ]:
도매가격 = pd.read_csv('../data/도매가/종합/도매_가격_완성본_13_21년도.csv', thousands=',',index_col = 0)

In [ ]:
도매가격.rename(columns={'가격':'도매가격'}, inplace = True)

## 도매 비어있는 날짜 추가 및 결측치 보간

### 기간 내 날짜 전체 생성

In [ ]:
start_date = pd.to_datetime('2013-01-01') ## 시작 날짜
end_date = pd.to_datetime('2021-12-31') ## 마지막 날짜
dates = pd.date_range(start_date,end_date,freq='D')
dates

In [ ]:
dates = pd.DataFrame(dates)
dates

In [ ]:
dates = pd.DataFrame(dates)
for i in range(0,4):
    dates = dates.append(dates.loc[:3286], ignore_index = True)
dates

In [ ]:
dates.columns = ['날짜']

In [ ]:
dates = dates.sort_values(by = '날짜').reset_index(drop=True)

In [ ]:
## 전 구간 전체지역 데이터프레임 생성

dates["지역"] = ['서울','대전','대구','부산','광주']*3287
dates

### 도매가 보간

In [ ]:
도매가격['날짜'] = pd.to_datetime(도매가격['날짜'], format='%Y-%m-%d')
도매가격.info()

### 겹치는 날짜 처리를 위한 거래량 분석

In [ ]:
## datetime str로 변경

도매가격["날짜"] = 도매가격["날짜"].dt.strftime('%Y-%m-%d')
도매가격.info()

In [ ]:
도매가격.dropna(inplace=True)
도매가격.info()

In [ ]:
도매가격.reset_index(drop = True, inplace = True)

In [ ]:
## 같은 날 다른 품명의 날짜 추출

date_list = []
for idx in 도매가격.index:
    point = 도매가격.iloc[idx]["날짜"]
    item = 도매가격.iloc[idx]["품명"]
    region = 도매가격.iloc[idx]["지역"]
    if len(도매가격[(도매가격['날짜'] == point) & (도매가격['지역'] == region) & (도매가격['품명'] != item)]) != 0:
        if pd.isna(item):
            print(도매가격.iloc[idx])
        else:
            date_list.append(point)

In [ ]:
date_list

In [ ]:
temp = list(set(date_list))
new_date = sorted(temp)

In [ ]:
res_date = new_date

In [ ]:
df_temp = pd.read_csv("../data/거래량/13_21년도_5대시장_거래량_종합.csv", index_col = 0)
df_temp

In [ ]:
## 겹치는 날짜에서 거래량 max인 품종의 품명을 추출해 리스트화

item_list = []
for date in res_date:
    df = df_temp[df_temp["날짜"] == date]
    vol = df.groupby('품명').sum()
    top = vol['거래량'].max()
    item_list.append(vol[vol['거래량'] == top].index)
item_list

In [ ]:
우세품 = pd.DataFrame(item_list, columns = ["품명"])
우세품

In [ ]:
우세품[우세품['품명'].isna()]

In [ ]:
## 거래 데이터가 없는 경우 앞 내용과 동일하게 채움 

우세품.loc[[95,96,141],["품명"]] = '가을'
우세품.loc[129,["품명"]]= '봄'

In [ ]:
## 겹치는 품목이 있는 날짜 프레임 생성

dup_date = pd.DataFrame(res_date, columns=['날짜'])
dup_date

In [ ]:
우세거래품 = pd.concat([dup_date,우세품], axis = 1)

In [ ]:
## 제거할 인덱스(우세품이 아닌 도매가데이터) 리스트 생성

idx_ls = []
for idx in 우세거래품.index:
    date = 우세거래품.iloc[idx,0]
    item = 우세거래품.iloc[idx,1]
    print(date,item)
    drop_idx = 도매가격[(도매가격.날짜 == date) & (도매가격.품명 != item)].index
    for item in drop_idx:
        idx_ls.append(item)
    print('---------------------------------------------------------')

In [ ]:
도매중복수정 = 도매가격.drop(idx_ls).reset_index(drop = True)
도매중복수정

In [ ]:
dates['날짜'] = dates['날짜'].dt.strftime('%Y-%m-%d')
dates.info()

In [ ]:
## 전체 구간에 현재 보유데이터 결합
temp = pd.merge(dates,도매중복수정, on=['날짜','지역'], how='left')
temp

In [ ]:
## 각 지역별로 분리

seoul_d = temp[temp.지역 == '서울'].reset_index(drop = True)
daejeon_d = temp[temp.지역 == '대전'].reset_index(drop = True)
daegu_d = temp[temp.지역 == '대구'].reset_index(drop = True)
busan_d = temp[temp.지역 == '부산'].reset_index(drop = True)
gwangju_d = temp[temp.지역 == '광주'].reset_index(drop = True)

# 전처리 - padding by ffill

In [ ]:
## 지역별로 나누어 가격 보간

seoul_p = copy.deepcopy(seoul_d)
daejeon_p = copy.deepcopy(daejeon_d)
daegu_p = copy.deepcopy(daegu_d)
busan_p = copy.deepcopy(busan_d)
gwangju_p = copy.deepcopy(gwangju_d)
pad_ls = [seoul_p, daejeon_p, daegu_p, busan_p, gwangju_p]

for value in pad_ls:
    temp = value.도매가격.fillna(method='ffill',inplace = True)
    print(value[value.도매가격.isna()])

In [ ]:
## 시작날짜 결측치 전날(2012-12-31)과 다음날 평균으로 대체

pr = [13000,13000,12000,12000,11500]
for item, value in zip(pad_ls,pr):
    item.iloc[0,3] = value

In [ ]:
## 품명보간

for item in pad_ls:
    item.interpolate(method = 'pad', limit=30, inplace=True) # 최근 값으로 보간
    item.fillna('가을', inplace= True)#시작날짜 품명 보간

### 데이터셋 파일 저장

In [ ]:
seoul_p.to_csv(path_or_buf='../data/도매가/보간/13_21_서울_도매가보간.csv', encoding = 'utf-8-sig')
daejeon_p.to_csv(path_or_buf='../data/도매가/보간/13_21_대전_도매가보간.csv', encoding = 'utf-8-sig')
daegu_p.to_csv(path_or_buf='../data/도매가/보간/13_21_대구_도매가보간.csv', encoding = 'utf-8-sig')
busan_p.to_csv(path_or_buf='../data/도매가/보간/13_21_부산_도매가보간.csv', encoding = 'utf-8-sig')
gwangju_p.to_csv(path_or_buf='../data/도매가/보간/13_21_광주_도매가보간.csv', encoding = 'utf-8-sig')